In [1]:
import pandas as pd
import numpy as np
import os
import numpy
import matplotlib.pyplot as plt
import SimpleITK
import itertools
import sys
import torch
from torchvision import transforms
from PIL import Image
from matplotlib import cm
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

from pathlib import Path

SOURCE_PATH = Path(os.getcwd()) / "src"

if SOURCE_PATH not in sys.path:
    sys.path.append(SOURCE_PATH)

from src.deeplab.modelling import deeplab_finetuning

%load_ext autoreload
%autoreload 2

In [2]:
path = Path(os.getcwd()) / "data_extracted"
type_to_use = "flair"

deeplab_finetuning(
    path,
    type_to_use,
    "seg",
    "models",
    model_exp_name="flair_20e_mobilenet_mse_noclass",
    pretrained=False,
    epochs=20,
    batch_size=16,
)

Model Created
Parameters defined
Dataloaders created
Epoch 1/20
----------


100%|██████████| 17/17 [01:24<00:00,  4.94s/it]


Train Loss: 0.0347


100%|██████████| 5/5 [00:08<00:00,  1.73s/it]


Test Loss: 0.0033
{'epoch': 1, 'Train_loss': 0.03466802090406418, 'Test_loss': 0.0033428757451474667, 'Train_f1_score': 0.06566540629430768, 'Test_f1_score': 0.0075118919288753375}
Epoch 2/20
----------


...

Test Loss: 0.0000
{'epoch': 19, 'Train_loss': 0.0014287525555118918, 'Test_loss': 3.9265934901777655e-05, 'Train_f1_score': 0.608418995227681, 'Test_f1_score': 0.2640590243092698}
Epoch 20/20
----------


100%|██████████| 17/17 [01:18<00:00,  4.64s/it]


Train Loss: 0.0015


 80%|████████  | 4/5 [00:06<00:01,  1.52s/it]C:\Users\ajayp\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1570: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
100%|██████████| 5/5 [00:06<00:00,  1.30s/it]

Test Loss: 0.0000
{'epoch': 20, 'Train_loss': 0.0015257748309522867, 'Test_loss': 4.0499562601326033e-05, 'Train_f1_score': 0.6176557583268872, 'Test_f1_score': 0.30259024587744254}
Training complete in 30m 31s
Lowest Loss: 0.000034
